In [1]:
import requests
import pprint
import json
import pandas as pd
import os as os1
import numpy as np
import pymongo
from pymongo import MongoClient


def get_country_list():
    country_metadata = pd.read_csv('Metadata_Country.csv')
    country_df = pd.DataFrame(country_metadata, columns =['Country_Code','Region','IncomeGroup','SpecialNotes','TableName'] )
    country_df['Country_Code'].str.strip()
    country_df['TableName'].str.strip()    
    return country_df

# Get Current GDP
def get_gdp_country(code):
    api_str_1 = 'http://api.worldbank.org/v2/country/'
    api_fetch = api_str_1 + code + '/indicator/NY.GDP.MKTP.CD?format=json'
    r = requests.get(api_fetch)
    gdp_country_js = r.json()
    gdp_country_l = gdp_country_js[1]
    return gdp_country_l

# Get GDP Per Capita
def get_gdp_pcp_country(code):
    api_str_1 = 'http://api.worldbank.org/v2/country/'
    api_fetch = api_str_1 + code + '/indicator/NY.GNP.PCAP.PP.CD?format=json'
    r = requests.get(api_fetch)
    gdp_country_js = r.json()
    gdp_country_l = gdp_country_js[1]
    return gdp_country_l


# Create and insert into database : "GDP_STG":
def insert_data_stg(name,gdp_cur_name, gdp_pci_name,GDP_CURRENT_LST,GNI_PPP_LST):
    client = MongoClient(host="localhost", port=27017)
    db = name
    mydb = client[db]
    collection_GDPC = gdp_cur_name #'SRC_STG_GDP_CUR'
    collection_GDPPCI = gdp_pci_name #'SRC_STG_GDP_PCI'
    mycol_GDPC =  mydb[collection_GDPC]
    mycol_GDPPCI= mydb[collection_GDPPCI]
    for item in GDP_CURRENT_LST:    
        x = mycol_GDPC.insert_many(item)
    for item in GNI_PPP_LST:
        x = mycol_GDPPCI.insert_many(item)
        
def main():
    # initializing empty lists to store GDP (current us$) and Gross National Income( Purchasing power parity)
    GDP_CURRENT_LST = []
    GNI_PPP_LST     = []
    
#     defining the db name and staging table names
    db_name = 'GDP_STG'
    gdp_cur_name = 'SRC_STG_GDP_CUR'
    gdp_pci_name = 'SRC_STG_GDP_PCI'
    
    
    country_list = get_country_list()
    for ind in country_list.index  :
        code = country_list['Country_Code'][ind]
        new_GDP = get_gdp_country(code)
        new_GNI = get_gdp_pcp_country(code)
        GDP_CURRENT_LST.append(new_GDP)
        GNI_PPP_LST.append(new_GNI)
    
    insert_data_stg(db_name,gdp_cur_name,gdp_pci_name,GDP_CURRENT_LST,GNI_PPP_LST)
    

if __name__ == '__main__':
    main()
    
  